In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
df = pd.read_csv('processed_data/all_data.csv')
df.head()

,CPF,Potencial Bruto,Raciocínio,Social,Motivacional,Cultura pontuação,atributo-Capacidade analítica,atributo-Pensamento conceitual,atributo-Reflexão,atributo-Pensamento criativo,...,perfil-Sociabilidade,perfil-Facilitação,perfil-Flexibilidade,perfil-Estabilidade emocional,perfil-Ambição,perfil-Iniciativa,perfil-Assertividade,perfil-Tomada de riscos,Área,AveragePerformance
0,678.042.935-18,53.65,65.70,37.65,56.75,32.33,59.67,49.44,63.08,56.26,...,23.77,41.95,55.59,78.32,60.14,41.95,8.74,55.59,Operações,2.50
1,089.324.175-12,NaN,60.78,NaN,NaN,35.13,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Logística,2.00
2,587.416.093-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Financeiro,2.00
3,375.410.982-13,22.46,15.71,5.67,56.75,23.36,11.43,18.25,13.14,16.54,...,91.86,32.86,46.50,46.50,32.86,51.05,32.86,41.95,Operações,2.75
4,652.134.809-42,NaN,56.15,NaN,NaN,62.03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pessoas,1.75
